In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

In [2]:
# read files
turn_over = pd.read_csv("../../Data/Turnover_Factor.csv")
financial_factor = pd.read_csv("../../Data/financial_quality_factor.csv")
stock_price = pd.read_csv("../../Data/Raw_Stock_Data.csv")
stock_industry = pd.read_csv("../../Data/stock_industry.csv")


In [3]:
# Get necessary data: stock price, market value, net return, net asset, industry and floatshare
floatshare = turn_over.loc[:, ["Date", "Index", "floatShares"]]

financial_factor = financial_factor.loc[:, ["date", "index", "BS_TOT_ASSET", "TOTAL_EQUITY", "RETURN_ON_ASSET"]].fillna(0)
financial_factor = financial_factor.rename(columns={"BS_TOT_ASSET": "total_asset", "TOTAL_EQUITY": "equity", "RETURN_ON_ASSET": "return_asset", "date":"Date","index":"Index"})
financial_factor["net return"] = financial_factor["total_asset"]*financial_factor["return_asset"]/100

stocks = stock_industry.rename(columns={"Stocks": "Index"})

stock_price = stock_price.loc[stock_price["Date"]>"2021-01-01"]
stock_price = stock_price[["Date","Index","Adj Close"]]
stock_price = stock_price.reset_index()
stock_price = stock_price.loc[:, ["Date", "Index", "Adj Close"]]

In [5]:
# Merge all tables to get EP, BP and industry
EP_BP_table = pd.merge(stock_price, floatshare, on=["Date", "Index"])
EP_BP_table["market value"] = EP_BP_table["floatShares"] * EP_BP_table["Adj Close"] / 1000000
EP_BP_table = EP_BP_table.merge(stocks, on="Index")
EP_BP_table = EP_BP_table.merge(financial_factor, on = ["Date", "Index"])
EP_BP_table['EP'] = EP_BP_table['net return'] / EP_BP_table['market value'] 
EP_BP_table['BP'] = EP_BP_table['equity'] / EP_BP_table['market value']
EP_BP_table = EP_BP_table.loc[:, ["Date", "Index", "EP", "BP", "industry"]]

,Date,Index,EP,BP,industry
0,2021-01-04,FTNT,0.025711,0.041258,Technology
1,2021-01-05,FTNT,0.026404,0.042370,Technology
2,2021-01-06,FTNT,0.027098,0.043483,Technology
3,2021-01-07,FTNT,0.026125,0.041923,Technology
4,2021-01-08,FTNT,0.025263,0.040539,Technology
...,...,...,...,...,...
263731,2022-07-25,AIH,0.000000,0.000000,Healthcare
263732,2022-07-26,AIH,0.000000,0.000000,Healthcare
263733,2022-07-27,AIH,0.000000,0.000000,Healthcare
263734,2022-07-28,AIH,0.000000,0.000000,Healthcare


In [6]:
# fill 0 with mean of history data
stock_index = EP_BP_table.loc[:, "Index"].unique()
for index in stock_index:
    print(index)
    EP_BP_table.loc[EP_BP_table["Index"]==index, "EP"] = EP_BP_table.loc[EP_BP_table["Index"]==index, "EP"].replace(0, np.nan)
    EP_BP_table.loc[EP_BP_table["Index"]==index, "BP"] = EP_BP_table.loc[EP_BP_table["Index"]==index, "BP"].replace(0, np.nan)
    EP_BP_table.loc[EP_BP_table["Index"]==index, "EP"] = EP_BP_table.loc[EP_BP_table["Index"]==index, "EP"].fillna(EP_BP_table.loc[EP_BP_table["Index"]==index, "EP"].mean())
    EP_BP_table.loc[EP_BP_table["Index"]==index, "BP"] = EP_BP_table.loc[EP_BP_table["Index"]==index, "BP"].fillna(EP_BP_table.loc[EP_BP_table["Index"]==index, "BP"].mean())

FTNT
LAND
GDYN
REG
ODFL
SYNA
PRFT
IMKTA
METC
PTSI
CLMT
MCBS
GAIN
SBNY
KRUS
LKQ
COKE
EXLS
PCB
INMD
WIRE
CCRN
PAYX
IRMD
AEHR
RELL
RRR
AFBI
INTU
CCB
MSFT
GOOGL
TBK
DOGZ
GOOD
BWFG
GOOG
NVDA
ATLC
BBCP
BCYC
CG
NVMI
CACC
STX
TW
ACLS
AVGO
ORLY
KFRC
LMST
LAMR
LNTH
CSCO
TSCO
MRBK
CDW
PSEC
SCVL
ASO
BNTX
HTBI
RICK
MXL
JYNT
AOSL
EXTR
KZR
SKIN
CPLP
SBAC
WSC
WRLD
CRAI
CAMT
CPRX
CRNX
CTHR
FANG
OPBK
ONEW
SITM
CROX
DVAX
SALM
SIMO
MYRG
RBB
PDSB
HFBL
ASML
AMAT
MRNA
LRFC
OMCL
NVEE
UFPI
DDOG
CRTO
CYBE
MRVL
AVID
PFMT
INSE
PI
CLBK
ZS
CASH
FRBA
TWNK
TBBK
SFM
LOB
CWST
PFG
RDWR
UPST
VCTR
AMBA
POOL
SGRY
BRY
TNDM
RETO
SCHL
RMBS
PVBC
GPP
CYTK
ESQ
PLBY
EQBK
MEDP
MBIN
VNOM
ZBRA
KRNT
SPT
AAPL
LEGH
MRUS
NTAP
SYNH
ISEE
HIMX
ICLR
HAYN
PLAB
LPLA
ENTA
JAKK
FNKO
BCRX
TEAM
SGH
PFSW
PETZ
SNPS
ESTA
KLAC
DSKE
ANGO
HUBG
STLD
SISI
GLAD
FFNW
TH
VBTX
BSVN
MORN
CTRN
HBCP
PTMN
RBCAA
SLAB
FFIV
MGYR
ZNTL
QFIN
MRAM
IRIX
SPFI
IDYA
LEGN
VECO
LBC
FAST
VRNT
ICFI
TROW
HONE
CONN
ULTA
HLIT
PZZA
HVBC
SGLY
CNNB
PRAA
HCCI
ISRG
MRVI
HMST
VALU
LQDA

In [7]:
# kick stocks that all BP and EPs are 0
missing_stocks = EP_BP_table[EP_BP_table['EP'].isna()]["Index"].unique()
remaining_stocks = list(set(EP_BP_table["Index"].unique()) - set(missing_stocks))
EP_BP_table = EP_BP_table[EP_BP_table['Index'].isin(remaining_stocks)]

In [34]:
# store EP, BP and industry factors
EP_BP_table.to_csv("../../Data/EP_BP_industry_Factor.csv", index=False)

In [42]:
# store missing stocks
missing = pd.DataFrame(missing_stocks, columns=["Index"])
missing.to_csv("../../Data/missing_stocks.csv", index=False)